In [3]:
# Импортируем библиотеки
import pandas as pd
import datetime
from helpful import *
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Считываем данные из файла
df = pd.read_excel('data/data.xlsx')

In [5]:
# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])

In [6]:
# Удаляем месяц, так как он не несет никакой информации
df = df.drop('month', axis=1)

# Удалим строки с merchant_name, которое встречается меньше 6 раз в df
df = df.groupby('merchant_name').filter(lambda x : len(x) > 5)
df.head(200)

,merchant_name,day,cashback
8350,#SEKTA,2022-04-18 00:00:00,347.49
8351,#SEKTA,2022-04-19 00:00:00,366.19
8352,#SEKTA,2022-04-20 00:00:00,142.50
8353,#SEKTA,2022-04-21 00:00:00,222.66
8354,#SEKTA,2022-04-22 00:00:00,90.23
...,...,...,...
9398,1С Интерес,2022-05-12 00:00:00,822.69
9399,1С Интерес,2022-05-13 00:00:00,693.33
9400,1С Интерес,2022-05-14 00:00:00,1419.64
9401,1С Интерес,2022-05-15 00:00:00,770.77


In [7]:
# Переведем столбец day формата "YYYY-MM-DD hh:mm:ss" в столбец с результатом get_date
df['day'] = df['day'].apply(get_date)
df['weekday'] = df['day'].apply(get_weekday)
df['is_weekend'] = df['weekday'].apply(lambda x : 1 if x in [5, 6] else 0)

In [8]:
# Заполним пропуски в столбце cashback средним значением
df['cashback'] = df['cashback'].fillna(df['cashback'].mean())

# Если между двумя строками разница 2 дня, то добавим строку между ними с пропущенными значениями
for i in range(len(df) - 1):
    if get_days_diff(df.iloc[i]['day'], df.iloc[i + 1]['day']) == 2:
        df = df._append({
            'merchant_name' : df.iloc[i]['merchant_name'], 
            'day' : get_next_day(df.iloc[i]['day']),
            'cashback' : (df.iloc[i]['cashback'] + df.iloc[i + 1]['cashback']) / 2, 
            'weekday' : get_weekday(get_next_day(df.iloc[i]['day'])),
            'is_weekend' : 1 if get_weekday(get_next_day(df.iloc[i]['day'])) in [5, 6] else 0},
            ignore_index=True)

# Отсортируем данные по именам и возрастании даты
df = df.sort_values(by=['merchant_name', 'day'])

In [9]:
# Для каждого значения merchant_name построим график cashback
'''
for merchant_name in df['merchant_name'].unique():
    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')
    plt.title(merchant_name)
    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')
    plt.close()
'''

"\nfor merchant_name in df['merchant_name'].unique():\n    sns.lineplot(data=df[df['merchant_name'] == merchant_name], x='day', y='cashback')\n    plt.title(merchant_name)\n    plt.savefig(f'graphs/{merchant_name.replace('*', ' ')}.png')\n    plt.close()\n"

In [10]:
# Сохраним df как clear_data.xlsx
'''
df.to_excel('data/clear_data.xlsx', index=False) 
'''

"\ndf.to_excel('data/clear_data.xlsx', index=False) \n"

In [11]:
df

,merchant_name,day,cashback,weekday,is_weekend
0,#SEKTA,2022-04-18,347.49,0,0
1,#SEKTA,2022-04-19,366.19,1,0
2,#SEKTA,2022-04-20,142.50,2,0
3,#SEKTA,2022-04-21,222.66,3,0
4,#SEKTA,2022-04-22,90.23,4,0
...,...,...,...,...,...
27401,интернет-магазин Технопарк,2022-07-27,26123.62,2,0
27402,интернет-магазин Технопарк,2022-07-28,25921.13,3,0
27403,интернет-магазин Технопарк,2022-07-29,35976.24,4,0
27404,интернет-магазин Технопарк,2022-07-30,32105.00,5,1
